In [14]:
import pandas as pd
from transformers import pipeline
import numpy as np
from tqdm import tqdm

books = pd.read_csv("books_with_categories.csv")

In [2]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k = None)

Device set to use cpu


In [3]:
classifier("I love this")

[[{'label': 'joy', 'score': 0.9845667481422424},
  {'label': 'surprise', 'score': 0.004927210509777069},
  {'label': 'sadness', 'score': 0.004531434271484613},
  {'label': 'neutral', 'score': 0.0034753174986690283},
  {'label': 'anger', 'score': 0.0013875322183594108},
  {'label': 'disgust', 'score': 0.0007134047336876392},
  {'label': 'fear', 'score': 0.0003984911891166121}]]

In [4]:
classifier("I hate this")

[[{'label': 'anger', 'score': 0.6354441046714783},
  {'label': 'disgust', 'score': 0.2896914482116699},
  {'label': 'sadness', 'score': 0.04677379131317139},
  {'label': 'neutral', 'score': 0.01836824044585228},
  {'label': 'fear', 'score': 0.004434023052453995},
  {'label': 'surprise', 'score': 0.0034457857254892588},
  {'label': 'joy', 'score': 0.0018427461618557572}]]

In [5]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.729602038860321},
  {'label': 'neutral', 'score': 0.14038637280464172},
  {'label': 'fear', 'score': 0.06816219538450241},
  {'label': 'joy', 'score': 0.047942470759153366},
  {'label': 'anger', 'score': 0.009156357496976852},
  {'label': 'disgust', 'score': 0.002628476358950138},
  {'label': 'sadness', 'score': 0.0021221644710749388}],
 [{'label': 'neutral', 'score': 0.44937166571617126},
  {'label': 'disgust', 'score': 0.273590624332428},
  {'label': 'joy', 'score': 0.10908307135105133},
  {'label': 'sadness', 'score': 0.09362711012363434},
  {'label': 'anger', 'score': 0.04047827422618866},
  {'label': 'surprise', 'score': 0.02697024680674076},
  {'label': 'fear', 'score': 0.006879054941236973}],
 [{'label': 'neutral', 'score': 0.6462171673774719},
  {'label': 'sadness', 'score': 0.2427321970462799},
  {'label': 'disgust', 'score': 0.04342259466648102},
  {'label': 'surprise', 'score': 0.02830052375793457},
  {'label': 'joy', 'score': 0.01421149168

In [7]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_score(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [12]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [ ]:
emotion_scores

{'anger': [np.float64(0.06413359940052032),
  np.float64(0.6126187443733215),
  np.float64(0.06413359940052032),
  np.float64(0.3514831066131592),
  np.float64(0.08141248673200607),
  np.float64(0.23222537338733673),
  np.float64(0.5381839871406555),
  np.float64(0.06413359940052032),
  np.float64(0.3006693422794342),
  np.float64(0.06413359940052032)],
 'disgust': [np.float64(0.273590624332428),
  np.float64(0.3482849895954132),
  np.float64(0.10400664061307907),
  np.float64(0.15072253346443176),
  np.float64(0.18449537456035614),
  np.float64(0.7271742224693298),
  np.float64(0.15585489571094513),
  np.float64(0.10400664061307907),
  np.float64(0.2794814109802246),
  np.float64(0.17792727053165436)],
 'fear': [np.float64(0.9281687140464783),
  np.float64(0.9425276517868042),
  np.float64(0.9723208546638489),
  np.float64(0.36070701479911804),
  np.float64(0.09504341334104538),
  np.float64(0.05136280134320259),
  np.float64(0.747428297996521),
  np.float64(0.4044966399669647),
  np.

In [15]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])  

100%|██████████| 5260/5260 [19:18<00:00,  4.54it/s]


In [16]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [17]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646217,0.967158,0.729602,9780002005883
1,0.612619,0.348285,0.942528,0.704421,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766,9780006178736
3,0.351483,0.150723,0.360707,0.251881,0.732685,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078766,9780006280934
...,...,...,...,...,...,...,...,...
5255,0.148208,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656,9788172235222
5256,0.064134,0.114383,0.051363,0.400262,0.883198,0.111690,0.227765,9788173031014
5257,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5258,0.064134,0.104007,0.459269,0.759456,0.951104,0.368111,0.078766,9788185300535


In [18]:
books = pd.merge(books, emotions_df, on="isbn13")

In [19]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_category,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928169,0.932798,0.646217,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348285,0.942528,0.704421,0.887939,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150723,0.360707,0.251881,0.732685,0.111690,0.078766
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656
5256,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400262,0.883198,0.111690,0.227765
5257,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5258,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759456,0.951104,0.368111,0.078766


In [20]:
books.to_csv("books_with_emotions.csv", index=False)